 Итак, вы познакомились с основными методами построения рекомендательных систем, и теперь настало время закрепить полученные знания на практике. В предыдущем модуле мы начали строить РС для сервиса чтения статей CI&T DeskDrop. В этом юните мы продолжим работу над ней.  

Примечание. Если у вас не сохранился код, который мы использовали ранее, вы можете найти его в ноутбуке RecSys_unit6.

Так как теперь вам известны более сложные алгоритмы построения рекомендательных систем, начнём с них.

Для начала необходимо построить матрицу, в которой по столбцам будут находиться id статей, по строкам — id пользователей, а на пересечениях строк и столбцов — оценка взаимодействия пользователя со статьёй. Если взаимодействия не было, в соответствующей ячейке должен стоять ноль.

In [1]:
import pandas as pd
import numpy as np
import math

In [3]:
articles_df = pd.read_csv('shared_articles.zip')
articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']

In [5]:
interactions_df = pd.read_csv('users_interactions.csv')

In [6]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

In [7]:
event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

In [8]:
interactions_df['eventStrength'] = interactions_df.eventType.apply(lambda x: event_type_strength[x])

In [9]:
users_interactions_count_df = (
    interactions_df
    .groupby(['personId', 'contentId'])
    .first()
    .reset_index()
    .groupby('personId').size())

users_with_enough_interactions_df = \
    users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['personId']]

In [10]:
interactions_from_selected_users_df = interactions_df.loc[np.in1d(interactions_df.personId,
            users_with_enough_interactions_df)]

In [11]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId']).eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index().set_index(['personId', 'contentId'])
)
interactions_full_df['last_timestamp'] = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId'])['timestamp'].max()
)
        
interactions_full_df = interactions_full_df.reset_index()

In [12]:
from sklearn.model_selection import train_test_split

split_ts = 1475519545
interactions_train_df = interactions_full_df.loc[interactions_full_df.last_timestamp < split_ts].copy()
interactions_test_df = interactions_full_df.loc[interactions_full_df.last_timestamp >= split_ts].copy()


In [13]:
# создаем таблицу сопряженности
ct = pd.crosstab(columns=interactions_train_df.contentId, # в колонках таблицы располагаем номера статей
                 index=interactions_train_df.personId, # в столбцах пользователей
                 values=interactions_train_df.eventStrength, # значения на пересечении
                 aggfunc='sum' # агрегирующая функция для значений
                 )
ct.fillna(0, inplace=True)
display(ct)

contentId,-1006791494035379303,-1021685224930603833,-1022885988494278200,-1024046541613287684,-1033806831489252007,-1038011342017850,-1039912738963181810,-1046621686880462790,-1051830303851697653,-1055630159212837930,...,9217155070834564627,921770761777842242,9220445660318725468,9222265156747237864,943818026930898372,957332268361319692,966067567430037498,972258375127367383,980458131533897249,98528655405030624
personId,,,,,,,,,,,,,,,,,,,,,
-1007001694607905623,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
-1032019229384696495,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,2.321928,0.0,0.0,0.0,0.0,0.0
-108842214936804958,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,2.0,0.0,0.0,0.0
-1130272294246983140,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0
-1160159014793528221,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953707509720613429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
983095443598229476,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
989049974880576288,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [14]:
# Задание 6.1
# 1 point possible (graded)
# Найдите оценку взаимодействия пользователя с ID -1032019229384696495 со статьёй с ID 943818026930898372. 
# Результат округлите до двух знаков после точки-разделителя.
print('оценкa взаимодействия пользователя с ID -1032019229384696495 со статьёй с ID 943818026930898372')
print(ct.loc[['-1032019229384696495'],['943818026930898372']].round(2).values[0][0])
# Примечание. Здесь и далее (пока не будет указано иное) необходимо работать с обучающей выборкой.

# answer
# 2.32  #   верно 

оценкa взаимодействия пользователя с ID -1032019229384696495 со статьёй с ID 943818026930898372
2.32


In [15]:
# Задание 6.2
# 1 point possible (graded)
# Найдите среднее арифметическое всех чисел в получившемся массиве. Результат округлите до трёх знаков после точки-разделителя.
print('среднее арифметическое всех чисел в получившемся массиве ', ct.mean().mean().round(3))

# Answer
# 0.017  # верно 

среднее арифметическое всех чисел в получившемся массиве  0.017


Перейдём к реализации коллаборативной фильтрации. Ранее мы делали это с помощью библиотеки surprise, однако это не всегда удобно, так как эта библиотека имеет ограниченное количество метрик для оценки качества и небольшой потенциал для более тонкой настройки алгоритма. Поэтому давайте попробуем реализовать алгоритмы коллаборативной фильтрации «с нуля». Такая практика применяется, если необходимо выстроить более сложную систему, чем могут предложить готовые модули. Кроме того, «ручная» реализация алгоритмов позволит лучше понять принцип их работы.

In [14]:
# Задание 6.3
# 1 point possible (graded)
# Постройте матрицу схожести. Для этого вычислите все попарные коэффициенты корреляции для матрицы, 
# полученной в предыдущем задании. Для каждой пары учитывайте только ненулевые значения 
# (так как нулевые обозначают отсутствие взаимодействия и не интересуют нас). 
# Выведите результат, полученный в ячейке с третьим индексом по строкам и сороковым — по столбцам. 
# Ответ округлите до двух знаков после точки-разделителя.

# Подсказка (1 из 1): Чтобы построить матрицу схожести, сначала создайте нулевую матрицу, 
# у которой количество строк и столбцов будет соответствовать количеству элементов в массиве из предыдущего задания.
# Затем реализуйте цикл в цикле (используйте цикл for): заполните ячейки коэффициентами корреляции, 
# если оба элемента с рассматриваемыми индексами не равны 0.

# Answer
# -0.33  # верно 

Матрица корреляции между пользователями будет квадратной так как мы сравниваем пользователей по их отношению к книгам, т.е. строки и столбцы этой матрицы равны количеству пользователей. Создадим нулевую матрицу такой размерности             
```my_matrix = np.zeros((len(ct), len(ct)))```.               
Нужно построить матрицу корреляции между пользователями, то есть в ```np.corrcoef``` нужно передать строки из массива, где строками являются пользователи, столбцами - книги, а значениями - оценка пользоваиеля. Поэтому создаем вложенный цикл - первый начинается с нулевого индекса(первого) пользователя, но не включает последний индекс, а второй наоборот, включает последний индекс, но начинается с первого индекса (второй пользователь). Это сделано для того, чтобы в итоговой матрице не появилось сравнение пользователя с самим собой.             
Так как нас интересует корреляция между пользователями только по не нулевым значениям - создаем маску:              
 ```mask_uv = (ct_arr[i] != 0) & (ct_arr[j] != 0)```,            
эта маска показывает True если оба пользователя поставили оценку книге. Далее записываем строки для текущей пары пользователей в переменные ```ratings_v``` и ```ratings_u```. Например для 3-го и 40-го пользователя одновременно оцененными оказалось только 4 книги, т.е. в наших переменных оказались массивы из 4 элементов. Далее находим для текущих пользователей коэффициент корреляции с помощью                 
```np.corrcoef(ratings_v, ratings_u)[0, 1]```.                  
 Полученный коэффициент корреляции вносим в нашу нулевую матрицу для соответствующих пользователей                
  ```my_matrix[i, j] = np.corrcoef(ratings_v, ratings_u)[0, 1]```.                 
Так как матрица семмитричная то ```my_matrix[j, i] = my_matrix[i, j] ```.


In [16]:
# Создаем квадратную матрицу нулей размером равным числу пользователей
my_matrix = np.zeros((len(ct), len(ct)))
my_matrix.shape

(1112, 1112)

In [17]:
ct_arr = ct.values
for i in range(len(ct_arr) - 1):
    for j in range(i + 1, len(ct_arr)):
        # print('i ', i)
        # print('j ', j)
        mask_uv = (ct_arr[i] != 0) & (ct_arr[j] != 0) 
        ratings_v = ct_arr[i, mask_uv] 
        ratings_u = ct_arr[j, mask_uv]
        my_matrix[i, j] = np.corrcoef(ratings_v, ratings_u)[0, 1] 
        my_matrix[j, i] = my_matrix[i, j]


/home/aubakirov/anaconda3/lib/python3.10/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/home/aubakirov/anaconda3/lib/python3.10/site-packages/numpy/core/_methods.py:182: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/home/aubakirov/anaconda3/lib/python3.10/site-packages/numpy/lib/function_base.py:2845: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/home/aubakirov/anaconda3/lib/python3.10/site-packages/numpy/lib/function_base.py:2704: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/home/aubakirov/anaconda3/lib/python3.10/site-packages/numpy/lib/function_base.py:2704: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
/home/aubakirov/anaconda3/lib/python3.10/site-packages/numpy/lib/function_base.py:2853: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None

In [18]:
display(my_matrix[3,40].round(2))

-0.33

Теперь у нас есть матрицы схожести пользователей. Их можно использовать для построения рекомендаций. Чтобы это сделать, надо реализовать следующий алгоритм.

Для каждого пользователя:

1. Найти пользователей с похожестью больше 0.
2. Для каждой статьи вычислить долю пользователей (среди выделенных на первом шаге), которые взаимодействовали со статьёй.
3. Порекомендовать статьи (не более 10) с наибольшими долями со второго шага (среди тех, которые пользователь ещё не видел).

In [18]:
# Задание 6.4
# 1 point possible (graded)
# Постройте рекомендательную систему по алгоритму, описанному выше. Найдите первую рекомендацию для строки 35 (если считать с нуля).

In [19]:
# Создаем словарь где ключами будут индексы пользователей, 
# а значениями список пользователей коэффициент корреляции с которыми больше 0
user_dict = {}

# В цикле перебираем все индексы пользователей
for i in range(1112):
# Создаем список для текущего индекса пользователя
    user_ls = []
# Повторно перебираем все индексы пользователей
    for j in range(1112):
# И сравниваем с текущим пользователем, при выполнении условия вносим индекс в список
        if my_matrix[i,j] > 0:
            user_ls.append(ct.index[j])
# Заносим полученный список в словарь под ключом, соответствующим текущему пользователю
    user_dict[ct.index[i]] = user_ls


In [20]:
# Делаем группировку по читателям и статьям, чтобы определить прочитанные статьи для пользователей
tr = interactions_train_df.groupby(['personId', 'contentId'], as_index=False)['eventStrength'].sum()

# Создаем массив ID пользователей
user_arr = ct.index

# В цикле проходим по ID пользователей и заполняем список списками из статей
ls = []
for user in user_arr:
    ls.append(tr[tr['personId']==user]['contentId'].to_list())

In [21]:
# Повторяем для тестовой выборки
ts = interactions_test_df.groupby(['personId', 'contentId'], as_index=False)['eventStrength'].sum()

ls_test = []
for user in user_arr:
    ls_test.append(ts[ts['personId']==user]['contentId'].to_list())

In [22]:
# Создаем датафрейм 
df_rec = pd.DataFrame({'userId':user_arr,'true_train':ls, 'true_test':ls_test})

In [23]:
# Создаем пустой список с рекомендациями
rec_ls = []

# В цикле проходим по всем пользователям
for user in user_arr:

# Фильтруем тренировочную выборку оставляя в ней только пользователей, которые похожи на текущего пользователя
    step_one = tr[tr['personId'].isin(user_dict[user])]

# Группируем результат фильтрации по статьям, а в значения заносим суммарную оценку статьи от похожих пользователей
    step_two = step_one.groupby(['contentId'], as_index=False)['eventStrength'].sum()

# Опять фильтруем результат оставляя только те статьи, которые текущий пользователь не читал
# .values вернет массив в котором будет наш список, поэтому используем обращение по индексу [0], чтобы до него добраться
    step_three = step_two[step_two['contentId'].isin(df_rec[df_rec['userId']==user]['true_train'].values[0])]
    
   
# Сортируем рейтинг статей по убыванию
    step_four = step_three.sort_values(['eventStrength'], ascending=False)

# Формируем список из первых 10 статей, это и будет наша рекомендация и заносим её в список
    rec_ls.append(step_four['contentId'].iloc[:10].to_list())


In [24]:
# Обновляем датафрейм
df_rec = pd.DataFrame({'userId':user_arr,'true_train':ls, 'true_test':ls_test, 'prediction_user_based':rec_ls })
display(df_rec.iloc[35]['prediction_user_based'][0])

'8657408509986329668'

***

In [28]:
# ЗАДАНИЕ 6.4

interactions = (
    interactions_train_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)

interactions['true_test'] = (
    interactions_test_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
)

interactions['true_test'] = [ [] if x is np.NaN else x for x in interactions['true_test'] ]

prediction_user_based = []
for i in range(len(my_matrix)):
    users_sim = my_matrix[i] > 0
    if not any(users_sim):
        prediction_user_based.append([])
    else:
        tmp_recommend = np.argsort(ct[users_sim].sum(axis=0))[::-1]
        tmp_recommend = ct.columns[tmp_recommend]
        recommend = np.array(tmp_recommend)[~np.in1d(tmp_recommend, interactions.iloc[i]["true_train"])][:10]
        prediction_user_based.append(list(recommend))
interactions['prediction_user_based'] = prediction_user_based
prediction_user_based[35][0]

'-5148591903395022444'

In [29]:
# ЗАДАНИЕ 6.5
def calc_precision(column):
    return ( interactions.apply(  lambda row:len(set(row['true_test']).intersection(
                set(row[column]))) /min(len(row['true_test']) + 0.001, 10.0), axis=1)).mean()

round(calc_precision('prediction_user_based'), 3)

0.005

In [31]:
# ЗАДАНИЕ 6.6
from scipy.linalg import svd

U, sigma, V = svd(ct)
U.max()

0.7071067811865483